In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Specify the directory name for saving the model
save_directory = "llama3"

# Load the tokenizer and model in half precision
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct", torch_dtype=torch.float16)

# Save the model and tokenizer to the 'llama3' directory
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

# Load the model and tokenizer from the 'llama3' directory
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModelForCausalLM.from_pretrained(save_directory, torch_dtype=torch.float16)

# Set device: GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the selected device
model = model.to(device)



/home/jiaping/anaconda3/envs/llama2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 12.04it/s]


In [2]:
# Define the prompt template for drone interaction (focused on steps only, no backward movement)
def get_drone_prompt(instruction, environment, obstacle):
    prompt_template = """
    ### Drone Instruction: {instruction}
    ### Environment: {environment_description}
    ### Obstacle Detected: {obstacle_type}
    ### Drone Action (steps only, avoid the closest obstacle first, move forward and avoid moving backward):
    """
    return prompt_template.format(
        instruction=instruction,
        environment_description=environment,
        obstacle_type=obstacle
    )

# Example scenario: the drone is flying in a room with obstacles
instruction = "Fly to the opposite side of the room and avoid all obstacles."
environment_description = "The room is 15x15 meters with several tables and chairs scattered around."
obstacle_type = "There is a chair 2 meters directly in front of you and a table 5 meters directly in front of you."

# Generate the prompt
formatted_prompt = get_drone_prompt(instruction, environment_description, obstacle_type)

# Encode the prompt using the tokenizer and move it to the same device as the model
inputs = tokenizer.encode(formatted_prompt, return_tensors="pt").to(device)

# Ensure model is in evaluation mode and run inference
model.eval()
with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_length=256,  # Adjust the max token limit as needed
        temperature=0.7,  # Adjust temperature for more deterministic or creative output
        top_p=1.0,
        top_k=0,  # Disable top-k filtering by setting top_k to 0
        repetition_penalty=1.0,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id  # Set pad token ID to avoid warnings
    )

# Decode the generated output back to text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Display the drone's action based on the generated response (steps only)
print("Drone's response to obstacle (steps only):")
print(generated_text)


/home/jiaping/anaconda3/envs/llama2/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Drone's response to obstacle (steps only):

    ### Drone Instruction: Fly to the opposite side of the room and avoid all obstacles.
    ### Environment: The room is 15x15 meters with several tables and chairs scattered around.
    ### Obstacle Detected: There is a chair 2 meters directly in front of you and a table 5 meters directly in front of you.
    ### Drone Action (steps only, avoid the closest obstacle first, move forward and avoid moving backward):
     1. Move forward by 2 meters to avoid the chair.
     2. Move forward by 5 meters to avoid the table.
     3. Move forward by 8 meters to reach the opposite side of the room.

### Solution (Python code):


```python
def fly_to_opposite_side(room_size, chair_position, table_position):
    """
    This function calculates the drone's movement instructions to fly to the opposite side of the room while avoiding obstacles.
    
    Args:
        room_size (tuple): The size of the room (length, width).
        chair_position (tuple): 